# Pavadintų objektų atpažinimas (NER)

Šis užrašų knygelė yra iš [AI for Beginners Curriculum](http://aka.ms/ai-beginners).

Šiame pavyzdyje išmoksime, kaip apmokyti NER modelį naudojant [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) duomenų rinkinį iš Kaggle. Prieš tęsiant, prašome atsisiųsti [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) failą į dabartinį katalogą.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Duomenų rinkinio paruošimas

Pradėsime nuo duomenų rinkinio įkėlimo į duomenų rėmelį. Jei norite sužinoti daugiau apie Pandas naudojimą, apsilankykite [pamokoje apie duomenų apdorojimą](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) mūsų [Duomenų mokslo pradedantiesiems](http://aka.ms/datascience-beginners) vadove.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Gaukime unikalius žymenis ir sukurkime paieškos žodynus, kuriuos galime naudoti žymenims konvertuoti į klasės numerius:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Dabar turime padaryti tą patį su žodynu. Dėl paprastumo sukursime žodyną, neatsižvelgdami į žodžių dažnį; realiame gyvenime galbūt norėsite naudoti Keras vektorizatorių ir apriboti žodžių skaičių.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Turime sukurti sakinių duomenų rinkinį mokymui. Pereikime per originalų duomenų rinkinį ir atskirkime visus atskirus sakinius į `X` (žodžių sąrašus) ir `Y` (žymų sąrašą):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Kad būtų paprasčiau, visas sakinius užpildysime 0 žetonais iki maksimalaus ilgio. Realiame gyvenime galėtume naudoti protingesnę strategiją ir užpildyti sekas tik viename mini pakete.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Apibrėžiant žymų klasifikavimo tinklą

Naudosime dviejų sluoksnių dvikryptį LSTM tinklą žymų klasifikavimui. Kad galėtume taikyti tankųjį klasifikatorių kiekvienam paskutinio LSTM sluoksnio išėjimui, naudosime `TimeDistributed` konstrukciją, kuri tą patį tankųjį sluoksnį pritaiko kiekvienam LSTM išėjimui kiekviename žingsnyje:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Atkreipkite dėmesį, kad čia aiškiai nurodome `maxlen` mūsų duomenų rinkiniui – jei norime, kad tinklas galėtų apdoroti kintamo ilgio sekas, turime būti šiek tiek sumanesni, kai apibrėžiame tinklą.

Dabar apmokykime modelį. Siekiant greičio, treniruosime tik vieną epochą, tačiau galite pabandyti treniruoti ilgesnį laiką. Taip pat galite atskirti dalį duomenų rinkinio kaip mokymo duomenų rinkinį, kad galėtumėte stebėti validacijos tikslumą.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Rezultato testavimas

Dabar pažiūrėkime, kaip mūsų subjektų atpažinimo modelis veikia su pavyzdiniu sakiniu:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Svarbiausia

Net ir paprastas LSTM modelis rodo pakankamai gerus rezultatus NER užduotyje. Tačiau, norint pasiekti žymiai geresnių rezultatų, verta naudoti didelius iš anksto apmokytus kalbos modelius, tokius kaip BERT. Kaip apmokyti BERT NER užduočiai naudojant Huggingface Transformers biblioteką, aprašyta [čia](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
